<a href="https://colab.research.google.com/github/Berenice2018/DeepLearning/blob/master/Conversion_PyTorch_to_Tensorflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# mount google drive
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


### PyTorch to ONNX

In [6]:
import torch.onnx
import torchvision


help(torch.onnx.export)

Help on function export in module torch.onnx:

export(*args, **kwargs)



In [0]:
# Standard ImageNet input - 3 channels, 224x224,
# values don't matter as we care about network structure.
# But they can also be real inputs.

dummy_input = torch.randn(1, 3, 224, 224)

# Obtain your model, it can be also constructed in your script explicitly
model = torchvision.models.squeezenet1_0(pretrained=True)

# Invoke export
base_path = './gdrive/My Drive/Colab Notebooks/Fer-dataset/'
export_path = base_path + "ONNX/squeezenet_test.onnx"

torch.onnx.export(model, dummy_input, export_path, verbose=True)


### Import ONNX to Tensorflow

In [0]:
!pip install onnx

In [0]:
!pip install onnx_tf

In [0]:
import onnx
import warnings
from onnx_tf.backend import prepare

warnings.filterwarnings('ignore') # Ignore all the warning messages in this tutorial

tf_model = onnx.load(export_path) # Load the ONNX file

tf_rep = prepare(tf_model) # Import the ONNX model to Tensorflow

In [22]:
# Now we have tf_rep, which is a python class containing four members: graph, inputs, outputs, and tensor_dict.

print(tf_rep.inputs) # Input nodes to the model
print('-----')
print(tf_rep.outputs) # Output nodes from the model
print('-----')
print(tf_rep.tensor_dict) # All nodes in the model

['0']
-----
['124']
-----
{'classifier.1.bias': <tf.Tensor 'Const:0' shape=(1000,) dtype=float32>, 'classifier.1.weight': <tf.Tensor 'Const_1:0' shape=(1000, 512, 1, 1) dtype=float32>, 'features.0.bias': <tf.Tensor 'Const_2:0' shape=(96,) dtype=float32>, 'features.0.weight': <tf.Tensor 'Const_3:0' shape=(96, 3, 7, 7) dtype=float32>, 'features.10.expand1x1.bias': <tf.Tensor 'Const_4:0' shape=(256,) dtype=float32>, 'features.10.expand1x1.weight': <tf.Tensor 'Const_5:0' shape=(256, 64, 1, 1) dtype=float32>, 'features.10.expand3x3.bias': <tf.Tensor 'Const_6:0' shape=(256,) dtype=float32>, 'features.10.expand3x3.weight': <tf.Tensor 'Const_7:0' shape=(256, 64, 3, 3) dtype=float32>, 'features.10.squeeze.bias': <tf.Tensor 'Const_8:0' shape=(64,) dtype=float32>, 'features.10.squeeze.weight': <tf.Tensor 'Const_9:0' shape=(64, 384, 1, 1) dtype=float32>, 'features.12.expand1x1.bias': <tf.Tensor 'Const_10:0' shape=(256,) dtype=float32>, 'features.12.expand1x1.weight': <tf.Tensor 'Const_11:0' shape=

### Prepare an image for inference

In [0]:
!pip uninstall tensorflow-gpu
!pip install --upgrade tensorflow

In [0]:
import tensorflow as tf
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession


config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)

In [16]:

import numpy as np
from IPython.display import display
from PIL import Image

path_to_img = base_path + 'fer2013/PrivateTest/2/PrivateTest_1162474.jpg'
#path_to_img = base_path + 'fer2013/testface-happy.jpg'

img = Image.open(path_to_img).resize((224, 224))

new_img = np.broadcast_to(img, (1, 3, 224, 224)).copy()
print(new_img.shape)

display(new_img) # show the image

img_ycbcr = img.convert("YCbCr")
img_y, img_cb, img_cr = img_ycbcr.split()


#test_face = np.asarray(img_y, dtype=np.float32)[np.newaxis, np.newaxis, :, :]

(224, 224)
(1, 3, 224, 224)


array([[[[ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         ...,
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202]],

        [[ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         ...,
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202]],

        [[ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         [ 18,  18,  18, ...,  88,  88,  88],
         ...,
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202],
         [ 16,  16,  16, ..., 202, 202, 202]]]], dtype=uint8)

In [17]:
# run the network
#dummy_face = torch.randn(1, 3, 224, 224)

print(new_img.shape)

test_face_run = tf_rep.run(new_img)._0

print(test_face_run.shape)

(1, 3, 224, 224)
(1, 1000)


### Tensorflow -.h5 to -.pb

In [0]:
from tensorflow.python.saved_model import builder as pb_builder

In [23]:
# load the model and save it as pb. 
# If the model is not in h5 format, convert it to h5 first.

#pre_model = tf.keras.models.load_model('path/to/your/model.model')
#pre_model.save('path/to/your/model.h5')

export_path_h5 = base_path + "ONNX/squeezenet_test.h5"

tf_rep.save(export_path_h5)

AttributeError: ignored